In [1]:
import copy
import os
import numpy as np
import pandas as pd

In [2]:
pattern_date = '2023-12-24'
root_dir = f'{"/".join(os.getcwd().split("/")[:-1])}'
format_anno_dir = f'{root_dir}/data/outputs/01.format_and_cv'
pose_output_dir = f'{root_dir}/data/outputs/02.pose_estimation'
highplace_output_dir = f'{root_dir}/data/outputs/04.highplace_detection/00.person_detection_add_label'
result_dir = f'{root_dir}/data/outputs/05.safetybelt_detection/00.safetybelt_detection_add_label'

In [3]:
annotation_all_df = pd.read_csv(f'{format_anno_dir}/annotation-{pattern_date}.csv')
annotation_person_df = pd.read_csv(f'{pose_output_dir}/person_eval_annotation-{pattern_date}.csv')
pred_person_df = pd.read_csv(f'{highplace_output_dir}/person_eval_pred_add_label-{pattern_date}.csv')

In [4]:
def calculate_intersection_per_label(pred_bbox, label_bbox):
    # バウンディングボックスの座標情報を取得
    x1_1, y1_1, x2_1, y2_1 = label_bbox
    x1_2, y1_2, x2_2, y2_2 = pred_bbox
    
    # 交差する領域を計算
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))
    intersection_area = x_intersection * y_intersection

    # 正解ラベルの面積
    area_label_bbox = (x2_1 - x1_1) * (y2_1 - y1_1)

    # 交差する領域の面積 / 正解ラベルの面積
    return intersection_area / area_label_bbox

In [5]:
def calculate_iou(box1, box2):
    # バウンディングボックスの座標情報を取得
    x1_1, y1_1, x2_1, y2_1 = box1
    x1_2, y1_2, x2_2, y2_2 = box2
    
    # 交差する領域を計算
    x_intersection = max(0, min(x2_1, x2_2) - max(x1_1, x1_2))
    y_intersection = max(0, min(y2_1, y2_2) - max(y1_1, y1_2))
    
    # 交差領域の面積と各バウンディングボックスの面積を計算
    intersection_area = x_intersection * y_intersection
    area_box1 = (x2_1 - x1_1) * (y2_1 - y1_1)
    area_box2 = (x2_2 - x1_2) * (y2_2 - y1_2)
    
    # IoUを計算
    iou = intersection_area / (area_box1 + area_box2 - intersection_area)
    
    return iou

## アノテーションした人に安全帯情報を付与

In [6]:
iou_threshold = 0.1
annotation_safetybelt_df = annotation_all_df[annotation_all_df["label"] == 'safety belt']
is_safetybelt_list = ['no-safetybelt' for _ in range(len(annotation_person_df))]
for i, person_info in enumerate(annotation_person_df[['unique_key', 'left', 'top', 'right', 'bottom']].values):
    unique_key = person_info[0]
    person_bbox = person_info[1:]
    safetybelt_bboxes = annotation_safetybelt_df[
        annotation_safetybelt_df['unique_key'] == unique_key
    ][['left', 'top', 'right', 'bottom']].values
    for safetybelt_bbox in safetybelt_bboxes:
        iou = calculate_iou(person_bbox, safetybelt_bbox)
        if iou > iou_threshold:
            is_safetybelt_list[i] = 'safetybelt'
annotation_person_df['label_safetybelt'] = is_safetybelt_list

In [7]:
print('学習データ内の評価用ピックアップデータのラベル件数')
display(pd.DataFrame(annotation_person_df[
    (annotation_person_df['unique_key'].str.contains('fixed-point-camera')) | 
    (annotation_person_df['unique_key'].str.contains('for-learning/2023-11-19-omaezaki-500')) | 
    (annotation_person_df['unique_key'].str.contains('for-learning/2023-11-23-mie-safetybelt'))
]['label_safetybelt'].value_counts()))

print('評価用データのラベル件数')
display(pd.DataFrame(annotation_person_df[
    (annotation_person_df['validation'] == 999)
]['label_safetybelt'].value_counts()))

学習データ内の評価用ピックアップデータのラベル件数


,count
label_safetybelt,
no-safetybelt,1714
safetybelt,346


評価用データのラベル件数


,count
label_safetybelt,
no-safetybelt,117
safetybelt,11


## 人検出した切り取り画像に、安全帯ラベルを付与

In [8]:
def make_safetybelt_cls(row, ratio_threshold=0.8, iou_threshold=0.3):
    img_key = row["unique_key"]
    pred_bbox = [row["left"], row["top"], row["right"], row["bottom"]]
    _annotation_person_df = copy.copy(annotation_person_df[annotation_person_df["unique_key"] == img_key])
    for gt_row in _annotation_person_df.values:
        if (
            calculate_intersection_per_label(pred_bbox, gt_row[3:7]) >= ratio_threshold
        ) and (
            calculate_iou(pred_bbox, gt_row[3:7]) >= iou_threshold
        ):
            return gt_row[-1]
    return "detection-miss"

In [9]:
pred_person_df["label_safetybelt"] = pred_person_df.apply(make_safetybelt_cls, axis=1)

In [10]:
print('学習データ内の評価用ピックアップデータのラベル件数')
display(pd.DataFrame(pred_person_df[
    (pred_person_df['unique_key'].str.contains('fixed-point-camera')) | 
    (pred_person_df['unique_key'].str.contains('for-learning/2023-11-19-omaezaki-500')) | 
    (pred_person_df['unique_key'].str.contains('for-learning/2023-11-23-mie-safetybelt'))
]['label_safetybelt'].value_counts()))

print('評価用データのラベル件数')
display(pd.DataFrame(pred_person_df[
    (pred_person_df['validation'] == 999)
]['label_safetybelt'].value_counts()))

学習データ内の評価用ピックアップデータのラベル件数


,count
label_safetybelt,
no-safetybelt,1458
safetybelt,342
detection-miss,205


評価用データのラベル件数


,count
label_safetybelt,
no-safetybelt,76
detection-miss,17
safetybelt,9


In [11]:
os.makedirs(result_dir, exist_ok=True)
annotation_person_df.to_csv(f'{result_dir}/safetybelt_eval_annotation_add_label-{pattern_date}.csv', index=False)
pred_person_df.to_csv(f'{result_dir}/safetybelt_eval_pred_add_label-{pattern_date}.csv', index=False)